# Create a user


In [ ]:
import requests
import pyotp
import time

base_url = "http://localhost:12437"
failures = 0
test_email = "tester7@devxt.com"


def register_user():
    login_response = requests.post(
        f"{base_url}/v1/user",
        json={
            "email": test_email,
            "first_name": "Test",
            "last_name": "Name",
            "company_name": "Test Company",
            "job_title": "Test Job Title",
        },
    )
    response = login_response.json()
    print(response)
    if "otp_uri" in response:
        mfa_token = response["otp_uri"].split("secret=")[1].split("&")[0]
        return mfa_token
    else:
        return response


while failures < 100:
    try:
        mfa_token = register_user()
        totp = pyotp.TOTP(mfa_token)
        break
    except Exception as e:
        print(e)
        failures += 1
        time.sleep(5)

## Confirm the user exists


In [ ]:
response = requests.get(
    f"{base_url}/v1/user/exists",
    params={"email": test_email},
)
print(response.json())

## Get magic link


In [ ]:
response = requests.post(
    f"{base_url}/v1/login",
    json={"email": test_email, "token": totp.now()},
)
response = response.json()
if "detail" in response:
    detail = response["detail"]
    if str(detail).startswith("http"):
        token = detail.split("token=")[1]
        print(f"Log in at {detail}")
    else:
        print(response)

## Update user


In [ ]:
response = requests.put(
    f"{base_url}/v1/user",
    json={
        "first_name": "Test",
        "last_name": "Man",
        "company_name": "Test Company",
        "job_title": "Test Job Title",
    },
    headers={"Authorization": f"{token}"},
)
print(response.json())

## Get user info


In [ ]:
response = requests.get(
    f"{base_url}/v1/user",
    headers={"Authorization": f"{token}"},
)
print(response.json())